In [1]:
import pandas as pd

from Bio import SeqIO
from Bio.Seq import Seq

In [2]:
class Trim:
    def __init__(self,main_fastq, barcode_fasta, length, output_dir, file_name):
        self.main_fastq = main_fastq
        self.barcode_fasta = barcode_fasta
        self.length = length
        self.output_dir = output_dir
        self.file_name = file_name

    def main_reader(self):
        n = []
        for record in SeqIO.parse(self.main_fastq, "fastq"):
            n.append(str(record.seq))
        return n

    def barcode_reader(self):
        barcodes = []
        for record in SeqIO.parse(self.barcode_fasta, "fasta"):
            barcodes.append(str(record.seq))
        return barcodes
        
    def trimmer(self):
        readed_main = self.main_reader()
        readed_barcodes = self.barcode_reader()

        trimmed = []
        for seq in readed_main:
            for barcode in readed_barcodes:
                index = seq.find(barcode)
                if not index == -1:
                    length = self.length
                    trimmed.append(seq[index:index + length])
        return trimmed

    def fasta_writer(self):
        trim = self.trimmer()
        path_file = self.output_dir + self.file_name
        file = open(path_file, 'w+')
        out = '\n'.join(['>line_' + str(i+1) + "\n" + j for i,j in enumerate(trim)])
        file.write(out)
        file.close()    


In [3]:
args = {
    'main_fastq': "/mnt/c/Users/pc/Downloads/barcod/qc_processed_barcode_1/N70_r1_qc_processed.fastq",
    'barcode_fasta': "/mnt/c/Users/pc/Downloads/barcod/qc_processed_barcode_1/b14.fasta",
    'output_dir': "/mnt/c/Users/pc/Downloads/barcod/qc_processed_barcode_1/",
    'file_name': 'trimmed.fasta',
    'length' : 48
    }

result = Trim(**args)

In [4]:
result.fasta_writer()

In [5]:
trimmonic = 1739877

my_trim = 1567482

In [8]:
from collections import Counter 

def eda(list):
    size = []
    for i in list:
        size.append(len(i))

    counts = Counter(size)

    return sorted(counts.items(), key=lambda i:i[1], reverse=True)

count_dict = eda(result.main_reader())

In [16]:
count_dict[:25]

[(147, 1570975),
 (146, 99795),
 (151, 20185),
 (145, 12717),
 (148, 11352),
 (150, 5719),
 (143, 2986),
 (149, 2914),
 (144, 2574),
 (54, 1776),
 (141, 1372),
 (139, 799),
 (142, 756),
 (135, 615),
 (133, 560),
 (137, 537),
 (132, 425),
 (131, 351),
 (114, 313),
 (136, 311),
 (134, 296),
 (128, 273),
 (138, 260),
 (124, 237),
 (127, 236)]